## Preprocess dataset for BERT pretraining

1. Prepare the dataset
2. Train a Tokenizer
3. Preprocess the dataset

Compute resource: AWS EC2 c5n.18xlarge instance

Reference: https://github.com/philschmid/deep-learning-habana-huggingface/blob/master/pre-training/pre-training-bert.ipynb

## 1. Prepare the dataset

In [3]:
# log into Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from huggingface_hub import HfApi
user_id = HfApi().whoami()["name"]

In [5]:
user_id

'delmeng'

The original BERT was pretrained on Wikipedia and BookCorpus dataset. Both datasets are available on the Hugging Face Hub and can be loaded with `datasets`.

https://huggingface.co/datasets/wikipedia  
https://huggingface.co/datasets/bookcorpus  

In [6]:
# note that this step is slow
from datasets import concatenate_datasets, load_dataset

bookcorpus = load_dataset("bookcorpus", split="train")
wikipedia = load_dataset("wikipedia", "20220301.en", split="train")
wikipedia = wikipedia.remove_columns([col for col in wikipedia.column_names if col != "text"])  # only keep the 'text' column

Generating train split:   0%|          | 0/74004228 [00:00<?, ? examples/s]

Downloading:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/20.3G [00:00<?, ?B/s]

In [ ]:
assert bookcorpus.features.type == wikipedia.features.type
raw_datasets = concatenate_datasets([bookcorpus, wikipedia])

In [ ]:
raw_datasets

## 2. Train a Tokenizer

In [ ]:
from tqdm import tqdm
from transformers import BertTokenizerFast

# repositor id for saving the tokenizer
tokenizer_id="bert-base-uncased-2023-pc-p4d"

# create a python generator to dynamically load the data
def batch_iterator(batch_size=10000):
    for i in tqdm(range(0, len(raw_datasets), batch_size)):
        yield raw_datasets[i : i + batch_size]["text"]

# create a tokenizer from existing one to re-use special tokens
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [ ]:
bert_tokenizer = tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
bert_tokenizer.save_pretrained("tokenizer")

In [ ]:
# push the tokenizer to Hugging Face Hub for later training our model
# you need to be logged into push the tokenizer
bert_tokenizer.push_to_hub(tokenizer_id)

## 3. Preprocess the dataset

Before we can get started with training our model, the last step is to pre-process/tokenize our dataset. We will use our trained tokenizer to tokenize our dataset and then push it to hub to load it easily later in our training. The tokenization process is also kept pretty simple, if documents are longer than 512 tokens those are truncated and not split into several documents.

In [ ]:
from transformers import AutoTokenizer
import multiprocessing

# load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(f"{user_id}/{tokenizer_id}")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")
num_proc = multiprocessing.cpu_count()
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

def group_texts(examples):
    tokenized_inputs = tokenizer(
       examples["text"], return_special_tokens_mask=True, truncation=True, max_length=tokenizer.model_max_length
    )
    return tokenized_inputs

# preprocess dataset
tokenized_datasets = raw_datasets.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
tokenized_datasets.features

As data processing function will we concatenate all texts from our dataset and generate chunks of tokenizer.model_max_length (512).

In [ ]:
from itertools import chain

# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= tokenizer.model_max_length:
        total_length = (total_length // tokenizer.model_max_length) * tokenizer.model_max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + tokenizer.model_max_length] for i in range(0, total_length, tokenizer.model_max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=num_proc)
# shuffle dataset
tokenized_datasets = tokenized_datasets.shuffle(seed=34)

print(f"the dataset contains in total {len(tokenized_datasets)*tokenizer.model_max_length} tokens")
# the dataset contains in total 3417216000 tokens

The last step before we can start with out training is to push our prepared dataset to the hub.

In [ ]:
# push dataset to hugging face
dataset_id=f"{user_id}/processed_bert_dataset"
tokenized_datasets.push_to_hub(dataset_id)

Processed dataset: https://huggingface.co/datasets/delmeng/processed_bert_dataset